In [1]:
import pandas as pd
import os
import networkx as nx
import numpy as np
pd.__version__

'0.25.3'

# Defining the different data path

In [2]:
# Data path containing all of the raw and processed data
data = '../Data/'
# Result path containing all the results from the analysisi
resultpath = '../Results/'
# ID for the PPI which year it's from 
PPI_ID = "2018_08"

# Loading all the tables

## Loading the raw counts

In [3]:
# Load the primary raw screen counts
raw_hipo_fec = pd.read_csv(os.path.join(data,'Screen', 'Raw', 'Raw_EggLaying_HpoRNAi.csv'))
raw_hipo_ova = pd.read_csv(os.path.join(data,'Screen', 'Raw', 'Raw_Ova_HpoRNAi.csv'))
raw_xRNAi_fec = pd.read_csv(os.path.join(data,'Screen', 'Raw', 'Raw_EggLaying.csv'))

# Load the prediction results
raw_hipo_fec_pred = pd.read_csv(os.path.join(data,'Screen', 'Raw', 'Raw_EggLaying_HpoRNAi_Pred.csv'))
raw_hipo_ova_pred = pd.read_csv(os.path.join(data,'Screen', 'Raw', 'Raw_Ova_HpoRNAi_Pred.csv'))
raw_xRNAi_fec_pred = pd.read_csv(os.path.join(data,'Screen', 'Raw', 'Raw_EggLaying_Pred.csv'))

# Remove Control from dataset
raw_hipo_fec = raw_hipo_fec[raw_hipo_fec['FbID'] != 'Control']
raw_hipo_ova = raw_hipo_ova[raw_hipo_ova['FbID'] != 'Control']
raw_xRNAi_fec = raw_xRNAi_fec[raw_xRNAi_fec['FbID'] != 'Control']

# Same on the prediction dataset
raw_hipo_fec_pred = raw_hipo_fec_pred[raw_hipo_fec_pred['FbID'] != 'Control']
raw_hipo_ova_pred = raw_hipo_ova_pred[raw_hipo_ova_pred['FbID'] != 'Control']
raw_xRNAi_fec_pred = raw_xRNAi_fec_pred[raw_xRNAi_fec_pred['FbID'] != 'Control']

# # Append the prediction count to the primary screen results
# raw_hipo_fec = raw_hipo_fec.append(raw_hipo_fec_pred)
# raw_hipo_ova = raw_hipo_ova.append(raw_hipo_ova_pred)
# raw_xRNAi_fec = raw_xRNAi_fec.append(raw_xRNAi_fec_pred)

# # Remove any duplicate
# raw_hipo_fec.drop_duplicates(inplace=True)
# raw_hipo_ova.drop_duplicates(inplace=True)
# raw_xRNAi_fec.drop_duplicates(inplace=True)

In [4]:
hipo_ova = pd.read_csv(os.path.join(data,'Screen', 'hipo_ova_clean.csv'))
xRNAi_fec = pd.read_csv(os.path.join(data,'Screen', 'xRNAi_fec_clean.csv'))
hipo_fec = pd.read_csv(os.path.join(data,'Screen', 'hipo_fec_clean.csv'))

hipo_ova_pred = pd.read_csv(os.path.join(data,'Screen', 'hipo_ova_clean_pred.csv'))
xRNAi_fec_pred = pd.read_csv(os.path.join(data,'Screen', 'xRNAi_fec_clean_pred.csv'))
hipo_fec_pred = pd.read_csv(os.path.join(data,'Screen', 'hipo_fec_clean_pred.csv'))

hipo_ova = hipo_ova[hipo_ova['FbID'] != 'Control']
xRNAi_fec = xRNAi_fec[xRNAi_fec['FbID'] != 'Control']
hipo_fec = hipo_fec[hipo_fec['FbID'] != 'Control']

hipo_ova_pred = hipo_ova_pred[hipo_ova_pred['FbID'] != 'Control']
xRNAi_fec_pred = xRNAi_fec_pred[xRNAi_fec_pred['FbID'] != 'Control']
hipo_fec_pred = hipo_fec_pred[hipo_fec_pred['FbID'] != 'Control']

In [5]:
# Calculate the mean for all datasets
mean_ova_gene = hipo_ova.groupby(['FbID'], as_index=False).mean()
mean_fec_gene = hipo_fec.groupby(['FbID', 'Condition'], as_index=False).mean()
mean_xRNAi_gene = xRNAi_fec.groupby(['FbID', 'Condition'], as_index=False).mean()

mean_ova_gene_pred = hipo_ova_pred.groupby(['FbID'], as_index=False).mean()
mean_fec_gene_pred = hipo_fec_pred.groupby(['FbID', 'Condition'], as_index=False).mean()
mean_xRNAi_gene_pred = xRNAi_fec_pred.groupby(['FbID', 'Condition'], as_index=False).mean()

# Calculate the std for ovariole number (only because the other datasets have only 1 measurement)
std_ova_gene = hipo_ova.groupby(['FbID']).std().reset_index()
std_ova_gene_pred = hipo_ova_pred.groupby(['FbID']).std().reset_index()

In [6]:
# mean_ova_gene = mean_ova_gene.append(mean_ova_gene_pred)
# mean_fec_gene = mean_fec_gene.append(mean_fec_gene_pred)
# mean_xRNAi_gene = mean_xRNAi_gene.append(mean_xRNAi_gene_pred)

In [7]:
# Here we select all the genes that were tested in the screen,
# because the first screen was Hipo RNAi EggLaying measurement, this dataset contains all the tested genes
screen_genes = mean_fec_gene['FbID'].unique()
screen_genes_pred = list(mean_fec_gene_pred['FbID'].unique())

In [8]:
modules = pd.read_csv(os.path.join(resultpath, "Modules_Table_{}.csv".format(PPI_ID)))

In [9]:
for fbid in modules[modules['SeedStatus'].str.contains('Connector')]['FbID'].values:
    if fbid not in screen_genes_pred:
        screen_genes_pred.append(fbid)

In [10]:
assert(len(mean_fec_gene['FbID'].unique()) == 463)
assert(len(mean_fec_gene_pred['FbID'].unique()) == 42)

## Loading gene names

In [11]:
names = pd.read_table(os.path.join(data,'GeneName.csv'))

## Loading the signaling pathway metadata

In [12]:
signaling = pd.read_csv(os.path.join(data,'signaling.csv'))

## Loading the PPI network

In [13]:
G = nx.read_graphml(os.path.join(data, 'PPIs', 'PPI_{}.graphml'.format(PPI_ID)))

## Loading the networks modules

In [14]:
# Modules computed in the notebook file: Seed-Connector
ova_module_G = nx.read_graphml(os.path.join(resultpath,'Ova_module_{}.graphml'.format(PPI_ID)))
fec_module_G = nx.read_graphml(os.path.join(resultpath,'Hpo_EggL_module_{}.graphml'.format(PPI_ID)))
xRNAi_module_G = nx.read_graphml(os.path.join(resultpath,'EggL_module_{}.graphml'.format(PPI_ID)))
core_module_G = nx.read_graphml(os.path.join(resultpath,'Core_module_{}.graphml'.format(PPI_ID)))

# The list of connector genes
connectors= pd.read_csv(os.path.join(resultpath,"ConnectorGeneList_{}.csv".format(PPI_ID)))

In [15]:
# Grab the list of genes for each modules
ova_module = ova_module_G.nodes()
fec_module = fec_module_G.nodes()
xRNAi_module = xRNAi_module_G.nodes()
core_module = core_module_G.nodes()

## Loading the network metrics

In [16]:
betweenness = pd.read_csv(os.path.join(data, "ScreenPPI_Betweenness.csv"))
closeness = pd.read_csv(os.path.join(data, "ScreenPPI_Closeness.csv"))
eigenvector = pd.read_csv(os.path.join(data, "ScreenPPI_Eigenvector.csv"))
degrees_cen = pd.read_csv(os.path.join(data, "ScreenPPI_DegreeCentrality.csv"))

# Creating the table

## Step 1: Make the list of genes

In [17]:
table = pd.DataFrame(screen_genes, columns=['FbID'])
table_pred = pd.DataFrame(screen_genes_pred, columns=['FbID'])

In [18]:
table = table.merge(raw_hipo_fec[['FbID', 'Condition']], how='left', on='FbID')
table = table.rename(columns={'Condition':'CG number'})

table_pred = table_pred.merge(raw_hipo_fec_pred[['FbID', 'Condition']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Condition':'CG number'})

In [19]:
table = table.merge(names, how='left', on='FbID')
table_pred = table_pred.merge(names, how='left', on='FbID')

## Step 2: Add the screen data for each genes

### Hippo RNAi Egg Laying

In [20]:
# Hippo RNAi Egg Laying screen egg counts
# First we merge the existing table using the FbID column for 1 to 1 matching.
# Then we rename that collumn for a unique name in the global database table
# Rinse and repeat for all values

table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 1'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_1_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_1_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 2 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_2_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_2_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 3'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_3_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_3_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 4 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_4_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_4_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 5'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_5_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_5_Egg_Count'})

table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Sum'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_All_Days_Egg_Sum_Zscore', 'Count':'HippoRNAi_EggL_All_Days_Egg_Sum_Count'})

# table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Sum'][['FbID', 'Batch']], how='left', on='FbID')
# table = table.rename(columns={'Batch':'HippoRNAi_EggL_Batch'})

In [21]:
table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Day 1'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'HippoRNAi_EggL_Day_1_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_1_Egg_Count'})
table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Day 2 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'HippoRNAi_EggL_Day_2_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_2_Egg_Count'})
table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Day 3'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'HippoRNAi_EggL_Day_3_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_3_Egg_Count'})
table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Day 4 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'HippoRNAi_EggL_Day_4_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_4_Egg_Count'})
table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Day 5'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'HippoRNAi_EggL_Day_5_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_5_Egg_Count'})

table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Sum'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'HippoRNAi_EggL_All_Days_Egg_Sum_Zscore', 'Count':'HippoRNAi_EggL_All_Days_Egg_Sum_Count'})

# table_pred = table_pred.merge(mean_fec_gene_pred[mean_fec_gene_pred['Condition'] == 'Sum'][['FbID', 'Batch']], how='left', on='FbID')
# table_pred = table_pred.rename(columns={'Batch':'HippoRNAi_EggL_Batch'})

### Egg Laying

In [22]:
# Egg Laying screen egg counts
# We use the same technic as above
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 1'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_1_Egg_Zscore', 'Count':'EggL_Day_1_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 2 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_2_Egg_Zscore', 'Count':'EggL_Day_2_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 3'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_3_Egg_Zscore', 'Count':'EggL_Day_3_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 4 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_4_Egg_Zscore', 'Count':'EggL_Day_4_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 5'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_5_Egg_Zscore', 'Count':'EggL_Day_5_Egg_Count'})

table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Sum'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_All_Days_Egg_Sum_Zscore', 'Count':'EggL_All_Days_Egg_Sum_Count'})

# table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Sum'][['FbID','Batch']], how='left', on='FbID')
# table = table.rename(columns={'Batch':'EggL_Batch'})

In [23]:
# Egg Laying screen egg counts
# We use the same technic as above
table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Day 1'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'EggL_Day_1_Egg_Zscore', 'Count':'EggL_Day_1_Egg_Count'})
table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Day 2 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'EggL_Day_2_Egg_Zscore', 'Count':'EggL_Day_2_Egg_Count'})
table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Day 3'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'EggL_Day_3_Egg_Zscore', 'Count':'EggL_Day_3_Egg_Count'})
table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Day 4 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'EggL_Day_4_Egg_Zscore', 'Count':'EggL_Day_4_Egg_Count'})
table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Day 5'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'EggL_Day_5_Egg_Zscore', 'Count':'EggL_Day_5_Egg_Count'})

table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Sum'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Z':'EggL_All_Days_Egg_Sum_Zscore', 'Count':'EggL_All_Days_Egg_Sum_Count'})

# table_pred = table_pred.merge(mean_xRNAi_gene_pred[mean_xRNAi_gene_pred['Condition'] == 'Sum'][['FbID','Batch']], how='left', on='FbID')
# table_pred = table_pred.rename(columns={'Batch':'EggL_Batch'})

### Ovariole Number

In [24]:
# Hippo RNAi Ovariole number counts screen
# We can merge all the data directly because we do not have many conditions in this table
table = table.merge(mean_ova_gene[['FbID', 'OvarioleNb', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Batch':'HippoRNAi_Ova_Batch', 
                              'OvarioleNb':'HippoRNAi_Ova_OvarioleNb_Mean_Count', 
                              'Z':'HippoRNAi_Ova_OvarioleNb_Mean_Zscore'})

table = table.merge(std_ova_gene[['FbID', 'OvarioleNb']], how='left', on='FbID')
table = table.rename(columns={'OvarioleNb':'HippoRNAi_Ova_OvarioleNb_Std_Count'})

In [25]:
table_pred = table_pred.merge(mean_ova_gene_pred[['FbID', 'OvarioleNb', 'Z']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'Batch':'HippoRNAi_Ova_Batch', 
                              'OvarioleNb':'HippoRNAi_Ova_OvarioleNb_Mean_Count', 
                              'Z':'HippoRNAi_Ova_OvarioleNb_Mean_Zscore'})

table_pred = table_pred.merge(std_ova_gene_pred[['FbID', 'OvarioleNb']], how='left', on='FbID')
table_pred = table_pred.rename(columns={'OvarioleNb':'HippoRNAi_Ova_OvarioleNb_Std_Count'})

In [26]:
# But we want to add the raw data to the table, so we extract the 20 columns from the raw counts
raw = raw_hipo_ova[raw_hipo_ova['FbID'].notnull()][['Fly 1', 'Fly 1.1', 'Fly 2', 'Fly 2.1', 
                                            'Fly 3', 'Fly 3.1', 'Fly 4', 'Fly 4.1', 
                                            'Fly 5', 'Fly 5.1', 'Fly 6', 'Fly 6.1',
                                            'Fly 7', 'Fly 7.1', 'Fly 8', 'Fly 8.1',
                                            'Fly 9','Fly 9.1', 'Fly 10', 'Fly 10.1',
                                                 'FbID']].groupby(['FbID']).mean().reset_index()

In [27]:
raw_pred = raw_hipo_ova_pred[raw_hipo_ova_pred['FbID'].notnull()][['Fly 1', 'Fly 1.1', 'Fly 2', 'Fly 2.1', 
                                            'Fly 3', 'Fly 3.1', 'Fly 4', 'Fly 4.1', 
                                            'Fly 5', 'Fly 5.1', 'Fly 6', 'Fly 6.1',
                                            'Fly 7', 'Fly 7.1', 'Fly 8', 'Fly 8.1',
                                            'Fly 9','Fly 9.1', 'Fly 10', 'Fly 10.1',
                                                 'FbID']].groupby(['FbID']).mean().reset_index()

In [28]:
# We merge them
table = table.merge(raw, how='left', on='FbID')
table_pred = table_pred.merge(raw_pred, how='left', on='FbID')


In [29]:
# And then we rename them to follow the naming scheme
table = table.rename(columns={'Fly 1'   : "HippoRNAi_Ova_OvarioleNb_Fly_1.1_Count",
                              'Fly 1.1' : "HippoRNAi_Ova_OvarioleNb_Fly_1.2_Count",
                              'Fly 2'   : "HippoRNAi_Ova_OvarioleNb_Fly_2.1_Count",
                              'Fly 2.1' : "HippoRNAi_Ova_OvarioleNb_Fly_2.2_Count",
                              'Fly 3'   : "HippoRNAi_Ova_OvarioleNb_Fly_3.1_Count",
                              'Fly 3.1' : "HippoRNAi_Ova_OvarioleNb_Fly_3.2_Count",
                              'Fly 4'   : "HippoRNAi_Ova_OvarioleNb_Fly_4.1_Count",
                              'Fly 4.1' : "HippoRNAi_Ova_OvarioleNb_Fly_4.2_Count",
                              'Fly 5'   : "HippoRNAi_Ova_OvarioleNb_Fly_5.1_Count",
                              'Fly 5.1' : "HippoRNAi_Ova_OvarioleNb_Fly_5.2_Count",
                              'Fly 6'   : "HippoRNAi_Ova_OvarioleNb_Fly_6.1_Count",
                              'Fly 6.1' : "HippoRNAi_Ova_OvarioleNb_Fly_6.2_Count",
                              'Fly 7'   : "HippoRNAi_Ova_OvarioleNb_Fly_7.1_Count",
                              'Fly 7.1' : "HippoRNAi_Ova_OvarioleNb_Fly_7.2_Count",
                              'Fly 8'   : "HippoRNAi_Ova_OvarioleNb_Fly_8.1_Count",
                              'Fly 8.1' : "HippoRNAi_Ova_OvarioleNb_Fly_8.2_Count",
                              'Fly 9'   : "HippoRNAi_Ova_OvarioleNb_Fly_9.1_Count",
                              'Fly 9.1' : "HippoRNAi_Ova_OvarioleNb_Fly_9.2_Count",
                              'Fly 10'  : "HippoRNAi_Ova_OvarioleNb_Fly_10.1_Count",
                              'Fly 10.1': "HippoRNAi_Ova_OvarioleNb_Fly_10.2_Count"
                             })

In [30]:
# And then we rename them to follow the naming scheme
table_pred = table_pred.rename(columns={'Fly 1'   : "HippoRNAi_Ova_OvarioleNb_Fly_1.1_Count",
                              'Fly 1.1' : "HippoRNAi_Ova_OvarioleNb_Fly_1.2_Count",
                              'Fly 2'   : "HippoRNAi_Ova_OvarioleNb_Fly_2.1_Count",
                              'Fly 2.1' : "HippoRNAi_Ova_OvarioleNb_Fly_2.2_Count",
                              'Fly 3'   : "HippoRNAi_Ova_OvarioleNb_Fly_3.1_Count",
                              'Fly 3.1' : "HippoRNAi_Ova_OvarioleNb_Fly_3.2_Count",
                              'Fly 4'   : "HippoRNAi_Ova_OvarioleNb_Fly_4.1_Count",
                              'Fly 4.1' : "HippoRNAi_Ova_OvarioleNb_Fly_4.2_Count",
                              'Fly 5'   : "HippoRNAi_Ova_OvarioleNb_Fly_5.1_Count",
                              'Fly 5.1' : "HippoRNAi_Ova_OvarioleNb_Fly_5.2_Count",
                              'Fly 6'   : "HippoRNAi_Ova_OvarioleNb_Fly_6.1_Count",
                              'Fly 6.1' : "HippoRNAi_Ova_OvarioleNb_Fly_6.2_Count",
                              'Fly 7'   : "HippoRNAi_Ova_OvarioleNb_Fly_7.1_Count",
                              'Fly 7.1' : "HippoRNAi_Ova_OvarioleNb_Fly_7.2_Count",
                              'Fly 8'   : "HippoRNAi_Ova_OvarioleNb_Fly_8.1_Count",
                              'Fly 8.1' : "HippoRNAi_Ova_OvarioleNb_Fly_8.2_Count",
                              'Fly 9'   : "HippoRNAi_Ova_OvarioleNb_Fly_9.1_Count",
                              'Fly 9.1' : "HippoRNAi_Ova_OvarioleNb_Fly_9.2_Count",
                              'Fly 10'  : "HippoRNAi_Ova_OvarioleNb_Fly_10.1_Count",
                              'Fly 10.1': "HippoRNAi_Ova_OvarioleNb_Fly_10.2_Count"
                             })

In [31]:
# Finally we finished entering the raw data into the table so we append both tables together prior to adding the metadata.
table = table.append(table_pred)

## Step 3: Adding the network metrics

In [32]:
# Merge and rename as above
table = table.merge(betweenness, how='left', on='FbID')
table = table.rename(columns={'Betweeness':'PPI_betweenness_centrality'})

In [33]:
table = table.merge(closeness, how='left', on='FbID')
table = table.rename(columns={'Closeness':'PPI_closeness_centrality'})

In [34]:
table = table.merge(eigenvector, how='left', on='FbID')
table = table.rename(columns={'EigenVector':'PPI_eigenvector_centrality'})

In [35]:
table = table.merge(degrees_cen, how='left', on='FbID')
table = table.rename(columns={'DegreeC':'PPI_degree_centrality'})

## Step 4: Adding the newtork modules

In [36]:
# We create 4 columns, where if a gene is found in a module it is 1 if not a 0
# np.where is key here, if condition == true, then X, else Y 
table['HippoRNAi_Ova_Module'] = np.where(table['FbID'].isin(ova_module), 1, 0)
table['HippoRNAi_EggL_Module'] = np.where(table['FbID'].isin(fec_module), 1, 0)
table['EggL_Module'] = np.where(table['FbID'].isin(xRNAi_module), 1, 0)
table['Core_Module'] = np.where(table['FbID'].isin(core_module), 1, 0)

In [37]:
meta_modules = np.array([np.where(table['FbID'].isin(ova_module), 1, 0), np.where(table['FbID'].isin(fec_module), 1, 0), np.where(table['FbID'].isin(xRNAi_module), 1, 0)])
modules = ["001","100","010","111","011","101","110","000"]
modules_leg = ["I","II","III","IV","V","VI","VII",""]
res = []
for el in meta_modules.T:
    i = modules.index(''.join([str(s) for s in el]))
    res.append(modules_leg[i])
table['Meta_Module'] = res

In [38]:
table.columns

Index(['FbID', 'CG number', 'NAME', 'SYMBOL', 'HippoRNAi_EggL_Day_1_Egg_Count',
       'HippoRNAi_EggL_Day_1_Egg_Zscore', 'HippoRNAi_EggL_Day_2_Egg_Count',
       'HippoRNAi_EggL_Day_2_Egg_Zscore', 'HippoRNAi_EggL_Day_3_Egg_Count',
       'HippoRNAi_EggL_Day_3_Egg_Zscore', 'HippoRNAi_EggL_Day_4_Egg_Count',
       'HippoRNAi_EggL_Day_4_Egg_Zscore', 'HippoRNAi_EggL_Day_5_Egg_Count',
       'HippoRNAi_EggL_Day_5_Egg_Zscore',
       'HippoRNAi_EggL_All_Days_Egg_Sum_Count',
       'HippoRNAi_EggL_All_Days_Egg_Sum_Zscore', 'EggL_Day_1_Egg_Count',
       'EggL_Day_1_Egg_Zscore', 'EggL_Day_2_Egg_Count',
       'EggL_Day_2_Egg_Zscore', 'EggL_Day_3_Egg_Count',
       'EggL_Day_3_Egg_Zscore', 'EggL_Day_4_Egg_Count',
       'EggL_Day_4_Egg_Zscore', 'EggL_Day_5_Egg_Count',
       'EggL_Day_5_Egg_Zscore', 'EggL_All_Days_Egg_Sum_Count',
       'EggL_All_Days_Egg_Sum_Zscore', 'HippoRNAi_Ova_OvarioleNb_Mean_Count',
       'HippoRNAi_Ova_OvarioleNb_Mean_Zscore',
       'HippoRNAi_Ova_OvarioleNb_Std_Coun

## Step 5: Adding Signaling pathways

In [39]:
# We need to make this tidy data, so we need to add one column per signaling patway, with a 0 or a 1 
# We iterate over all signaling pathway and add a column for each with 1 and 0s using the same np.where technic as above
# But we first need to make the list of FbID that have this signaling pathway 
# that is: pathway_genes = signaling[signaling['Sig'] == pathway]['FbID']
for pathway in signaling['Sig'].unique():
    pathway_genes = signaling[signaling['Sig'] == pathway]['FbID']
    table['{}_pathway'.format(pathway)] = np.where(table['FbID'].isin(pathway_genes), 1, 0)

## Step 6: Adding Connector genes

In [40]:
table['HpoOva_Connector'] = table['FbID'].isin(connectors[connectors['Module'] == 'Ova']['FbID'].values)
table['HpoEggL_Connector'] = table['FbID'].isin(connectors[connectors['Module'] == 'HpoFec']['FbID'].values)
table['EggL_Connector'] = table['FbID'].isin(connectors[connectors['Module'] == 'xRNAiFec']['FbID'].values)
table['Core_Connector'] = table['FbID'].isin(connectors[connectors['Module'] == 'Core']['FbID'].values)

## Step 7: Cleaning up

In [41]:
final_order = ['FbID',
'CG number',
'NAME',
'SYMBOL',
'HippoRNAi_EggL_Day_1_Egg_Count',
'HippoRNAi_EggL_Day_1_Egg_Zscore',
'HippoRNAi_EggL_Day_2_Egg_Count',
'HippoRNAi_EggL_Day_2_Egg_Zscore',
'HippoRNAi_EggL_Day_3_Egg_Count',
'HippoRNAi_EggL_Day_3_Egg_Zscore',
'HippoRNAi_EggL_Day_4_Egg_Count',
'HippoRNAi_EggL_Day_4_Egg_Zscore',
'HippoRNAi_EggL_Day_5_Egg_Count',
'HippoRNAi_EggL_Day_5_Egg_Zscore',
'HippoRNAi_EggL_All_Days_Egg_Sum_Count',
'HippoRNAi_EggL_All_Days_Egg_Sum_Zscore',
'EggL_Day_1_Egg_Count',
'EggL_Day_1_Egg_Zscore',
'EggL_Day_2_Egg_Count',
'EggL_Day_2_Egg_Zscore',
'EggL_Day_3_Egg_Count',
'EggL_Day_3_Egg_Zscore',
'EggL_Day_4_Egg_Count',
'EggL_Day_4_Egg_Zscore',
'EggL_Day_5_Egg_Count',
'EggL_Day_5_Egg_Zscore',
'EggL_All_Days_Egg_Sum_Count',
'EggL_All_Days_Egg_Sum_Zscore',
'HippoRNAi_Ova_OvarioleNb_Fly_1.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_1.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_2.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_2.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_3.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_3.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_4.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_4.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_5.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_5.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_6.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_6.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_7.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_7.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_8.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_8.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_9.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_9.2_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_10.1_Count',
'HippoRNAi_Ova_OvarioleNb_Fly_10.2_Count',
'HippoRNAi_Ova_OvarioleNb_Mean_Count',
'HippoRNAi_Ova_OvarioleNb_Std_Count',
'HippoRNAi_Ova_OvarioleNb_Mean_Zscore',
'PPI_betweenness_centrality',
'PPI_closeness_centrality',
'PPI_eigenvector_centrality',
'PPI_degree_centrality',
'HippoRNAi_Ova_Module',
'HippoRNAi_EggL_Module',
'EggL_Module',
'Core_Module',
'Meta_Module',
'HpoOva_Connector',
'HpoEggL_Connector',
'EggL_Connector',
'Core_Connector',
'FGF_pathway',
'VEGF_pathway',
'Toll_pathway',
'JNK_pathway',
'EGF_pathway',
'mTOR_pathway',
'FOXO_pathway',
'SHH_pathway',
'Hippo_pathway',
'JAK/STAT_pathway',
'Wnt_pathway',
'Notch_pathway',
'TGF B_pathway',
'MAPK_pathway']

In [42]:
table = table[final_order]

In [43]:
def round_centrality(x):
    return round(x, 10)

for col in [c for c in table.columns if 'PPI_' in c]:
    table[col] = table[col].apply(round_centrality)

In [44]:
for col in [c for c in table.columns if 'PPI_' in c]:
    table[col] = table[col].fillna('NotInPPI')

In [45]:
for col in [c for c in table.columns if '_Module' in c]:
    if not 'Meta' in col:
        PPIcheck = table['PPI_closeness_centrality'].values
        tmp = table[col].values.astype(str)
        for i in range(len(tmp)):
            if PPIcheck[i] == 'NotInPPI':
                tmp[i] = 'NotInPPI'
            else:
                if tmp[i] == '0':
                    tmp[i] = 'False'
                else:
                    tmp[i] = 'True'
        table[col] = tmp

In [46]:
for col in [c for c in table.columns if '_Connector' in c]:
    PPIcheck = table['PPI_closeness_centrality'].values
    tmp = table[col].values.astype(object)
    for i in range(len(tmp)):
        if PPIcheck[i] == 'NotInPPI':
            tmp[i] = "NotInPPI"
    table[col] = tmp

In [47]:
def round_zscore(x):
    return round(x, 4)

for col in [c for c in table.columns if '_Zscore' in c]:
    table[col] = table[col].apply(round_zscore)

In [48]:
# Adding the CG number for the missing connector
table.at[table[table['FbID'] == 'FBgn0027619'].index, 'CG number'] = "CG12131"

# Asserting that the data is correctly entered

In [49]:
# Test that the number of genes in the database is equal to the number of gene screened in the primary screen
assert(len(table) == len(screen_genes) + len(screen_genes_pred)) # -1 to get rid of the control ID
total_datapoints = len(screen_genes) + len(screen_genes_pred)

In [50]:
# Test that the number of values in each screen correspond to the raw table for each collumn
# We iterate over all the collumns
# For each collumns we define a test
# try the assertion
# if wrong then print an error message
for column in table.columns:
    if 'Module' in column:
        try:
            assert(len(table[table[column].notna()]) == total_datapoints)
        except:
            print("Discrepancy in column: {}".format(column))
    elif '_pathway' in column:
        try:
            assert(len(table[table[column].notna()]) == total_datapoints)
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'HippoRNAi_EggL' in column:
        try:
            assert(len(table[table[column].notna()]) == total_datapoints - 1)
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'EggL' in column and not "Connector" in column:
        try:
            assert(len(table[table[column].notna()]) == len(mean_xRNAi_gene['FbID'].unique()) + len(screen_genes_pred) - 1)
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'HippoRNAi_Ova' in column:
        try:
            assert(len(table[table[column].notna()]) == len(mean_ova_gene['FbID'].unique()) + len(screen_genes_pred) - 1)
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'Connector_Gene' == column:
        try:
            assert(len(table[table[column] == 1]) == len(connectors['FbID'].unique()))
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'PPI_' in column:
        try:
            assert(len(table[table[column] != 'NotInPPI']) ==  len(table[table['FbID'].isin(G.nodes())]))
        except:
            print("Discrepancy in column: {}".format(column))
    elif "CG" in column:
        try:
            assert(len(table[table[column].notna()]) == total_datapoints)
        except:
            print("Discrepancy in column: {}".format(column))
            

# Saving the table

In [51]:
table.to_csv(os.path.join(resultpath, "MasterTable.csv"), index=False)

In [87]:
len(table[table['HippoRNAi_Ova_Module'] == 'True']['FbID'])

81

In [85]:
len(table[table['Core_Module'] == 'True']['FbID'])

41

In [84]:
len(table[table['EggL_Module'] == 'True']['FbID'])

67

In [88]:
len(table[table['HippoRNAi_EggL_Module'] == 'True']['FbID'])

84

In [89]:
len(table[table['HippoRNAi_Ova_Module'] == 'True']['FbID'].unique())

77

In [86]:
len(table[table['Core_Module'] == 'True']['FbID'].unique())

37

In [83]:
len(table[table['EggL_Module'] == 'True']['FbID'].unique())

60

In [82]:
len(table[table['HippoRNAi_EggL_Module'] == 'True']['FbID'].unique())

76

In [90]:
len(table[table['HpoOva_Connector'] == True]['FbID'])

13

In [91]:
len(table[table['HpoOva_Connector'] == True]['FbID'].unique())

11

In [92]:
len(table[table['HpoEggL_Connector'] == True]['FbID'])

25

In [93]:
len(table[table['HpoEggL_Connector'] == True]['FbID'].unique())

18

In [94]:
len(table[table['EggL_Connector'] == True]['FbID'])

18

In [95]:
len(table[table['EggL_Connector'] == True]['FbID'].unique())

11

In [116]:
table[(table['EggL_All_Days_Egg_Sum_Zscore'].abs() > 5) & (table['PPI_closeness_centrality'] == 'NotInPPI')]

,FbID,CG number,NAME,SYMBOL,HippoRNAi_EggL_Day_1_Egg_Count,HippoRNAi_EggL_Day_1_Egg_Zscore,HippoRNAi_EggL_Day_2_Egg_Count,HippoRNAi_EggL_Day_2_Egg_Zscore,HippoRNAi_EggL_Day_3_Egg_Count,HippoRNAi_EggL_Day_3_Egg_Zscore,...,EGF_pathway,mTOR_pathway,FOXO_pathway,SHH_pathway,Hippo_pathway,JAK/STAT_pathway,Wnt_pathway,Notch_pathway,TGF B_pathway,MAPK_pathway


In [117]:
table[(table['HippoRNAi_EggL_All_Days_Egg_Sum_Zscore'].abs() > 5) & (table['PPI_closeness_centrality'] == 'NotInPPI')]

,FbID,CG number,NAME,SYMBOL,HippoRNAi_EggL_Day_1_Egg_Count,HippoRNAi_EggL_Day_1_Egg_Zscore,HippoRNAi_EggL_Day_2_Egg_Count,HippoRNAi_EggL_Day_2_Egg_Zscore,HippoRNAi_EggL_Day_3_Egg_Count,HippoRNAi_EggL_Day_3_Egg_Zscore,...,EGF_pathway,mTOR_pathway,FOXO_pathway,SHH_pathway,Hippo_pathway,JAK/STAT_pathway,Wnt_pathway,Notch_pathway,TGF B_pathway,MAPK_pathway
323,FBgn0037325,CG12147,-,CG12147,169.0,4.1758,83.0,1.3024,74.0,3.5152,...,0,0,0,0,0,0,1,0,0,0


In [115]:
table[(table['HippoRNAi_Ova_OvarioleNb_Mean_Zscore'].abs() > 2) & (table['PPI_closeness_centrality'] == 'NotInPPI')]

,FbID,CG number,NAME,SYMBOL,HippoRNAi_EggL_Day_1_Egg_Count,HippoRNAi_EggL_Day_1_Egg_Zscore,HippoRNAi_EggL_Day_2_Egg_Count,HippoRNAi_EggL_Day_2_Egg_Zscore,HippoRNAi_EggL_Day_3_Egg_Count,HippoRNAi_EggL_Day_3_Egg_Zscore,...,EGF_pathway,mTOR_pathway,FOXO_pathway,SHH_pathway,Hippo_pathway,JAK/STAT_pathway,Wnt_pathway,Notch_pathway,TGF B_pathway,MAPK_pathway
51,FBgn0002592,CG6104,"Enhancer of split m2, Bearded family member",E(spl)m2-BFM,25.0,-0.9201,5.0,-2.6049,4.0,-1.3096,...,0,0,0,0,0,0,0,1,0,0


In [110]:
table[(table['Core_Module'] == 'True') & (table['PPI_closeness_centrality'] == 'NotInPPI')]

,FbID,CG number,NAME,SYMBOL,HippoRNAi_EggL_Day_1_Egg_Count,HippoRNAi_EggL_Day_1_Egg_Zscore,HippoRNAi_EggL_Day_2_Egg_Count,HippoRNAi_EggL_Day_2_Egg_Zscore,HippoRNAi_EggL_Day_3_Egg_Count,HippoRNAi_EggL_Day_3_Egg_Zscore,...,EGF_pathway,mTOR_pathway,FOXO_pathway,SHH_pathway,Hippo_pathway,JAK/STAT_pathway,Wnt_pathway,Notch_pathway,TGF B_pathway,MAPK_pathway


In [114]:
table[(table['HippoRNAi_EggL_All_Days_Egg_Sum_Zscore'].abs() > 5)]

,FbID,CG number,NAME,SYMBOL,HippoRNAi_EggL_Day_1_Egg_Count,HippoRNAi_EggL_Day_1_Egg_Zscore,HippoRNAi_EggL_Day_2_Egg_Count,HippoRNAi_EggL_Day_2_Egg_Zscore,HippoRNAi_EggL_Day_3_Egg_Count,HippoRNAi_EggL_Day_3_Egg_Zscore,...,EGF_pathway,mTOR_pathway,FOXO_pathway,SHH_pathway,Hippo_pathway,JAK/STAT_pathway,Wnt_pathway,Notch_pathway,TGF B_pathway,MAPK_pathway
0,FBgn0000042,CG4027,Actin 5C,Act5C,0.0,-4.3294,0.0,-3.7259,0.0,-2.7062,...,0,0,0,0,1,0,0,0,0,0
4,FBgn0000047,CG5178,Actin 88F,Act88F,0.0,-4.7429,0.0,-3.5457,0.0,-2.5488,...,0,0,0,0,1,0,0,0,0,0
6,FBgn0000117,CG11579,armadillo,arm,0.0,-1.7265,0.0,-1.5121,0.0,-2.6317,...,0,0,0,0,0,0,1,0,0,0
16,FBgn0000259,CG15224,Casein kinase II beta subunit,CkIIbeta,1.0,-8.8669,0.0,-9.2968,0.0,-2.9382,...,0,0,0,0,0,0,1,0,0,0
20,FBgn0000352,CG1708,costa,cos,0.0,-4.3294,0.0,-3.7259,0.0,-2.7062,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,FBgn0035213,CG2199,-,CG2199,0.0,-4.7975,0.0,-4.3319,0.0,-6.2258,...,0,0,0,0,0,0,0,0,0,0
490,FBgn0035640,CG17498,mad2,mad2,54.0,-4.0274,39.0,-1.6130,36.0,-1.3093,...,0,0,0,0,0,0,0,0,0,0
491,FBgn0036136,CG6233,Ubiquitin fusion-degradation 1-like,Ufd1-like,0.0,-6.0226,0.0,-3.7674,0.0,-3.2410,...,0,0,0,0,0,0,0,0,0,0
493,FBgn0037874,CG4800,Translationally controlled tumor protein,Tctp,39.0,-5.0584,20.0,-1.6291,11.0,-2.3763,...,0,0,0,0,0,0,0,0,0,0


In [118]:
meta_module_G = nx.read_graphml(os.path.join(resultpath,'Meta_module_{}.graphml'.format(PPI_ID)))


In [122]:
connectors = []
for node in meta_module_G.nodes():
    if 'Connector' in meta_module_G.nodes[node]['Seed']:
        connectors.append(node)

In [124]:
len(connectors)

42

In [128]:
tab = table[(table['HpoEggL_Connector'] == True) | (table['HpoOva_Connector'] == True) | (table['EggL_Connector'] == True) | (table['Core_Connector'] == True)]['FbID'].unique()

In [129]:
len(tab)

43

In [131]:
set(connectors).difference(set(tab))

set()

In [132]:
for g in tab:
    if g not in connectors:
        print(g)

FBgn0003430
